In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

2022-01-03 12:21:58.337029: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

2022-01-03 12:22:05.841204: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-03 12:22:05.843867: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-03 12:22:05.900199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 12:22:05.901005: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-01-03 12:22:05.901056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-03 12:22:05.911323: I tensorflow/stream_executor/platform/defau

In [4]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [5]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [6]:
def create_model(base_model, dropout):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(dropout),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(dropout),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()
val_accs = []
for i in range(3):
    base_model = vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(48,48,3))
    if i == 0:
        model = create_model(base_model, 0.5)
        history_save_path = './results/dropout-05.txt'
    if i == 1:
        model = create_model(base_model, 0.25)
        history_save_path = './results/dropout-025.txt'
    if i == 2:
        model = create_model(base_model, 0.75)
        history_save_path = './results/dropout-075.txt'
    history = train(model, learning_rate, loss, num_epochs, batch_size)
    val_accs.append(history.history['val_model_acc'])
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

/userhome/cs/u3553423/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(
2022-01-03 12:22:09.721134: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3523239936 exceeds 10% of free system memory.
2022-01-03 12:22:12.899993: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-03 12:22:12.900865: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199845000 Hz
2022-01-03 12:22:12.936449: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


Epoch 1/40


2022-01-03 12:22:15.819209: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-03 12:22:16.464741: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


3983/3983 [==============================] - 329s 81ms/step - loss: 0.0325 - model_acc: 0.6525 - val_loss: 0.0164 - val_model_acc: 0.8016
Epoch 2/40
3983/3983 [==============================] - 333s 84ms/step - loss: 0.0152 - model_acc: 0.8159 - val_loss: 0.0142 - val_model_acc: 0.8194
Epoch 3/40
3983/3983 [==============================] - 327s 82ms/step - loss: 0.0115 - model_acc: 0.8548 - val_loss: 0.0135 - val_model_acc: 0.8267
Epoch 4/40
3983/3983 [==============================] - 325s 82ms/step - loss: 0.0091 - model_acc: 0.8836 - val_loss: 0.0129 - val_model_acc: 0.8340
Epoch 5/40
3983/3983 [==============================] - 342s 86ms/step - loss: 0.0074 - model_acc: 0.9050 - val_loss: 0.0125 - val_model_acc: 0.8439
Epoch 6/40
3983/3983 [==============================] - 311s 78ms/step - loss: 0.0061 - model_acc: 0.9198 - val_loss: 0.0128 - val_model_acc: 0.8358
Epoch 7/40
3983/3983 [==============================] - 335s 84ms/step - loss: 0.0052 - model_acc: 0.9324 - val_loss:

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()

with open('./results/history_base+flip+rotate20.txt') as file_pi:
    hist = pickle.load(file_pi)
plt.plot(epochs, hist['val_model_acc'], label = 'dropout=0')

plot_labels = ['dropout=0.5', 'dropout=0.25', 'dropout=0.75']
for i in range(3):
    plt.plot(epochs, val_accs[i], label = plot_labels[i])

plt.legend()
plt.show()
# save plot
fig.savefig('./plots/dropout.png')